In [ ]:
%pip install google-generativeai

In [8]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

In [9]:
# Or use `os.getenv('GEMINI_API_KEY')` to fetch an environment variable.
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
#GEMINI_API_KEY = 'un-api-valido'
genai.configure(api_key=GEMINI_API_KEY)

In [10]:
# ===================== CONFIGURACIÓN INICIAL =========================

# Definir las reglas por categoría
reglas = {
    "izquierda": [
        "corregir nombres y apellidos",
        "corregir ortografia",
        "verificar cohesión y coherencia del texto"
    ],"derecha": [
        "corregir nombres y apellidos",
        "corregir ortografia",
        "verificar cohesión y coherencia del texto"
    ],
    "neutral": [
        "corregir nombres y apellidos",
        "corregir ortografia",
        "verificar cohesión y coherencia del texto"
    ],
}

# distintos modelos que usamos por una cuestión de quoata
#model = genai.GenerativeModel("gemini-1.5-flash")
model = genai.GenerativeModel("gemini-2.0-flash-lite")
#model = genai.GenerativeModel("gemini-2.0-flash")
#model = genai.GenerativeModel("gemini-2.0-flash-exp")


In [11]:
# ===================== FUNCIONES =========================

def crear_prompt(texto, categoria):
    """Genera el prompt dinámico con las reglas e 
       indica que se informe qué cambios se realizaron."""

    instrucciones = "\n- " + "\n- ".join(reglas[categoria])
    prompt = (
        f"Corrige el siguiente texto aplicando estas reglas:{instrucciones}\n"
        f"Texto original:\n{texto}\n"
        "Texto corregido: solamente el texto corregido, las correcciones y cambios que realizaste "
        "escribilos en el archivo de log.\n"
    )
    return prompt


def procesar_texto_con_gemini(texto, categoria):
    """Crea el prompt y obtiene el texto corregido y las explicaciones de cambios."""

    prompt = crear_prompt(texto, categoria)
    print(f'cantidad de tokens procesados: {model.count_tokens(prompt)}')

    try:
        response = model.generate_content(prompt)
        # tratamos los bloqueos a veces por quota a veces por contenido no permitido
        if (response.prompt_feedback and response.prompt_feedback.block_reason 
            != genai.enums.BlockedReason.BLOCKED_REASON_UNSPECIFIED):
            print(f"Prompt blocked for reason: {response.prompt_feedback.block_reason}")
            return None, f"Prompt bloqueado por: {response.prompt_feedback.block_reason}" 
        elif not response.candidates:
             print("No candidates returned, but prompt was not explicitly blocked.")
             return None, "No se generaron candidatos para la respuesta."

        texto_respuesta = response.text.strip()

        # separamos el texto corregido de las explicaciones
        partes = texto_respuesta.split("Cambios realizados:")
        if len(partes) == 2:
            texto_corregido = partes[0].strip()
            cambios = partes[1].strip()
        else:
            texto_corregido = texto_respuesta
            cambios = "No se proporcionaron detalles de los cambios."

        return texto_corregido, cambios
    except Exception as e:
        print(f"An error occurred during API call: {e}")
        return None, f"Error en la llamada a la API: {e}"


def leer_archivo(ruta):
    with open(ruta, 'r', encoding='utf-8') as f:
        return f.read()


def guardar_archivo(ruta, contenido):
    with open(ruta, 'w', encoding='utf-8') as f:
        f.write(contenido)


def registrar_log(carpeta_salida, nombre_archivo, correccion, cambios):
    ruta_log = os.path.join(carpeta_salida, "log_correcciones.txt")
    with open(ruta_log, 'a', encoding='utf-8') as f:
        f.write(f"Archivo: {nombre_archivo} - Corrección: {correccion}\n")
        f.write(f"Cambios realizados: {cambios}\n\n")


def procesar_categoria(categoria):
    carpeta = categoria
    carpeta_salida = categoria + "_corr"

    # mkdir de carpeta de salida si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    archivos = [f for f in os.listdir(carpeta) if f.endswith(".txt")]

    if not archivos:
        print(f"No se encontraron archivos en {carpeta}")
        return 0

    cantidad = 0

    for archivo in archivos:
        ruta_entrada = os.path.join(carpeta, archivo)
        ruta_salida = os.path.join(carpeta_salida, archivo)

        print(f"Procesando {ruta_entrada}...")

        texto = leer_archivo(ruta_entrada)
        texto_corregido, cambios = procesar_texto_con_gemini(texto, categoria)

        # Check if the processing was successful (texto_corregido is not None)
        if texto_corregido is not None:
            guardar_archivo(ruta_salida, texto_corregido)
            # Generar log para correcciones exitosas
            correccion_resumen = f"Texto corregido aplicando reglas: {', '.join(reglas[categoria])}"
            registrar_log(carpeta_salida, archivo, correccion_resumen, cambios)

            print(f"Guardado en {ruta_salida}\n")
            cantidad += 1
        else:
            # Log the reason for not processing if texto_corregido is None
            registrar_log(carpeta_salida, archivo, "No procesado", cambios)
            print(f"No se pudo procesar {ruta_entrada}. Razón: {cambios}\n")
            # Continue to the next file without raising an error
            continue


    return cantidad


In [12]:
# ===================== EJECUCIÓN =========================

categorias = ["izquierda", "derecha", "neutral"]
conteo_total = {}

for categoria in categorias:
    print(f"\n=== Procesando categoría: {categoria} ===")
    cantidad = procesar_categoria(categoria)
    conteo_total[categoria] = cantidad

print("\n✅ Corrección finalizada para todas las categorías.\n")

# Mostrar conteo de correcciones
for categoria, cantidad in conteo_total.items():
    print(f"Archivos corregidos en {categoria}: {cantidad}")



=== Procesando categoría: izquierda ===
Procesando izquierda\🔴 editorial fernando rosso  la hora de la verdad.txt...
cantidad de tokens procesados: total_tokens: 2195

Guardado en izquierda_corr\🔴 editorial fernando rosso  la hora de la verdad.txt

Procesando izquierda\🔴 editorial fernando rosso  libertarianismo de estado.txt...
cantidad de tokens procesados: total_tokens: 1719

Guardado en izquierda_corr\🔴 editorial fernando rosso  libertarianismo de estado.txt

Procesando izquierda\🔴 editorial fernando rosso  menem y milei tragedia y farsa.txt...
cantidad de tokens procesados: total_tokens: 2218

Guardado en izquierda_corr\🔴 editorial fernando rosso  menem y milei tragedia y farsa.txt

Procesando izquierda\🚨 menem amenaza periodistas 🚨 editorial de roberto navarro.txt...
cantidad de tokens procesados: total_tokens: 1367

Guardado en izquierda_corr\🚨 menem amenaza periodistas 🚨 editorial de roberto navarro.txt

Procesando izquierda\🚨 nos despidieron a todos   editorial de ari lijalad

## Utils - Estas funciones no son impresindibles para la corrección de los textos

In [ ]:
# extraer en el directorio actual el contenido de archivos.zip

import zipfile

def extract_zip_files():
  """Extracts the contents of all .zip files in the current directory."""
  current_directory = os.getcwd()
  for filename in os.listdir(current_directory):
    if filename.endswith(".zip"):
      try:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
          zip_ref.extractall(current_directory)
          print(f"Extracted: {filename}")
      except zipfile.BadZipFile:
        print(f"Error: Could not extract {filename}. It might be a corrupted zip file.")
      except Exception as e:
        print(f"An error occurred while extracting {filename}: {e}")

# Call the function to perform the extraction
extract_zip_files()


In [7]:
# eliminar los archivos de la carpeta derecha,que se encuentran en la carpeta derecha_corr
# se puede usar para las otras categorías también

# Get the list of files in the "derecha_corr" folder
files_to_delete = os.listdir("derecha_corr")
#
## Iterate over the list and delete the corresponding files in the "derecha" folder
for file_name in files_to_delete:
    file_path_to_delete = os.path.join("derecha", file_name)
    if os.path.exists(file_path_to_delete):
        os.remove(file_path_to_delete)
        print(f"Deleted: {file_path_to_delete}")
    else:
        print(f"File not found in 'derecha': {file_path_to_delete}")
  

File not found in 'derecha': derecha\12 cuotas - editorial #bastababy 16052022.txt
File not found in 'derecha': derecha\a cristina no le importa nada y el presidente parece no tener destino.txt
File not found in 'derecha': derecha\a cristina, los límites de la república le molestan.txt
File not found in 'derecha': derecha\a medida - editorial #bastababy 21092022.txt
File not found in 'derecha': derecha\a todos - editorial #bastababy 2122022.txt
File not found in 'derecha': derecha\a uno lo crío yo - editorial #bastababy 23062022.txt
File not found in 'derecha': derecha\agua bendita - editorial #bastababy 05092022.txt
File not found in 'derecha': derecha\ahora, la política se hace la sorprendida con los planes sociales (porque llegaron las encuestas).txt
File not found in 'derecha': derecha\al final la vida sigue igual - editorial #bastababy 21042022.txt
File not found in 'derecha': derecha\al psiquiatrico i editorial de joni viale en ¿la ves.txt
File not found in 'derecha': derecha\alb

#### Utils colab

In [ ]:
# Used to securely store your API key
from google.colab import userdata

# Or use `os.getenv('GEMINI_API_KEY')` to fetch an environment variable.
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)


In [ ]:
# manterner la sesión activa de colab
import time

count = 0
while True:
  print(f"Keeping session alive: {count}")
  count += 1
  time.sleep(10)
